In [1]:
import numpy as np

In [2]:
def find_neighbors(q, x):
    if q in x:
        index = np.searchsorted(x, q)
        print("q is the collocation point with index ", index)
    elif q < x[0]:
        print("The query q is too small to be interpolated given x.")
    elif q > x[-1]:
        print("The query q is too big to be interpolated given x.")
    else:
        for i in range(len(x)):
            if q < x[i+1]:
                print("q = ", q, " lays between x_", i, "=", x[i], " and x_", i+1,"=", x[i+1])
                return i
                break

In [3]:
x = [0.0, 0.5, 1.0, 1.5, 2.0]
y = [0.0, 1.0, 0.0, -1.0, 0.0]

In [4]:
find_neighbors(0.25, x)

q =  0.25  lays between x_ 0 = 0.0  and x_ 1 = 0.5


0

In [5]:
# Gives now correct result for Q3, but different from matlab in Q4
def spline_interpolation(x, y, xq):
    n = len(x)
    assert len(x) == len(y), 'lists for x and y must have same length'
    
    i = find_neighbors(xq, x)
    print("i = ", i)
    
    h = np.zeros(n-1)
    a = np.zeros(n-2)
    b = np.zeros(n-1)
    c = np.zeros(n-1)
    d = np.zeros(n)

    for j in range(len(h)):
        h[j] = x[j+1] - x[j] # hi = xi+1 - xi
    print("h = ", h)

    for j in range(len(a)):
        a[j] = 2/h[j] + 2/h[j+1]
    print("a = ", a)

    for j in range(len(b)):
        b[j] = 1/h[j]
    print("b = ", b) 

    for j in range(len(c)):
        c[j] = (y[j+1] - y[j]) / (h[j] * h[j])
    print("c = ", c)

    for j in range(len(d)-2):
        d[j+1] = 3 * (c[j] + c[j+1])
    d[-1] = 2 * c[-1] + h[-1] / (h[-1] + h[-2]) * (c[-1] + c[-2]) # d' from not-a-knot-condition
    print("d = ", d)

    
    
    # Constructing coefficient matrix A needed to solve for the first derivatives of f
    A = np.zeros(shape=(n-1, n-1))
    
    # Boundary conditions to render system solvable
    # Boor's not-a-knot-condition
    a_nminus2_prime = 1/h[-2] + 1/h[-1]
    print("an-2' = ", a_nminus2_prime)

    b_nminus2_prime = 1/h[-1]
    print("bn-1' = ", b_nminus2_prime)
    
    A[-1, -2] = a_nminus2_prime;
    A[-1, -1] = b_nminus2_prime;

    
    
    # Boundary conditions: periodic spline
    # Fill first row of A according to periodic B.C.s
    A[0, 0] = a[0];
    A[0, 1] = b[1];
    A[0, -1] = b[0];
    
    # Filling rest of A starting from second row
    k = -1; # column of A
    for j in range(1, n-2):
        k += 1;
        A[j, k]   = b[j];
        A[j, k+1] = a[j];
        A[j, k+2] = b[j+1];


    print("A = ", A)

    df = np.zeros(n)
    df[1:] = np.linalg.solve(A, d[1:])
    df[0] = df[-1] # Periodic Boundary Conditions
    print("df = ", df)



    t = (xq - x[i]) / h[i]
    print("t = ", t)
    Q = y[i] * (1 - 3*t**2 + 2*t**3) + y[i+1] * ( 3*t**2 - 2*t**3) + h[i] * df[i] * (t - 2*t**2 + t**3) + h[i] * df[i+1] * (-t**2 + t**3)
    print("Q = ", Q)
    return Q


In [6]:
xq = 0.25
spline_interpolation(x, y, xq)

q =  0.25  lays between x_ 0 = 0.0  and x_ 1 = 0.5
i =  0
h =  [0.5 0.5 0.5 0.5]
a =  [8. 8. 8.]
b =  [2. 2. 2. 2.]
c =  [ 4. -4. -4.  4.]
d =  [  0.   0. -24.   0.   8.]
an-2' =  4.0
bn-1' =  2.0
A =  [[8. 2. 0. 2.]
 [2. 8. 2. 0.]
 [0. 2. 8. 2.]
 [0. 0. 4. 2.]]
df =  [ 5.33333333 -0.66666667 -2.66666667 -0.66666667  5.33333333]
t =  0.5
Q =  0.8749999999999999


0.8749999999999999

In [7]:
# Compare with master solution
0.875

0.875